# Урок 3. Построение надежных схем валидации решения, оптимизация целевых метрик

##### Импорт библиотек

In [1]:
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
# import catboost as cb
# import lightgbm as lgb

import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter("ignore")
# warnings.filterwarnings('ignore')

## Основное задание:
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы/минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - `IsFraud`. Рекомендуется использовать модели градиетного бустинга, реализация любая, гипепараметры любые.

**NB!** выборка `assignment_2_test.csv` - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!

Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit /train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [3]:
train = pd.read_csv('./data/assignment2_data/assignment_2_train.csv')
leaderboard = pd.read_csv('./data/assignment2_data/assignment_2_test.csv')
target = 'isFraud'

print("train.shape = {} rows, {} cols".format(*train.shape))
print("leaderboard.shape = {} rows, {} cols".format(*leaderboard.shape))

train.shape = 180000 rows, 394 cols
leaderboard.shape = 100001 rows, 394 cols


In [4]:
train.head(n=3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Задание 1. 
Сделать Hold-Out валидацию с разбиением, размер которого будет адекватным, по вашему мнению. Разбиение проводить по id-транзакции (`TransactionID`), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [5]:
numerical_features = train \
    .select_dtypes(include=[np.number]) \
    .drop(target, axis=1) \
    .columns \
    .tolist()

In [6]:
X, y = train[numerical_features].fillna(0), train[target]
X_lb, y_lb = leaderboard[numerical_features].fillna(0), leaderboard[target]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=False, random_state=42)

In [8]:
params = {
    "booster": "gbtree", "objective": "binary:logistic",
    "eval_metric": "auc", "learning_rate": 0.1, "n_estimators": 1000,
    "reg_lambda": 100, "max_depth": 4, "gamma": 10, "nthread": 6, "seed": 27
}

model_xgb_numeric = xgb.XGBClassifier(**params)
model_xgb_numeric.fit(X=x_train, y=y_train,
                      eval_set=[(x_train, y_train), (x_test, y_test)],
                      early_stopping_rounds=30,
                      eval_metric="auc",
                      verbose=20)

[0]	validation_0-auc:0.59781	validation_1-auc:0.61482
[20]	validation_0-auc:0.81385	validation_1-auc:0.82154
[40]	validation_0-auc:0.87183	validation_1-auc:0.84969
[60]	validation_0-auc:0.88592	validation_1-auc:0.86278
[80]	validation_0-auc:0.89107	validation_1-auc:0.86816
[100]	validation_0-auc:0.89542	validation_1-auc:0.86979
[120]	validation_0-auc:0.89897	validation_1-auc:0.87192
[140]	validation_0-auc:0.90179	validation_1-auc:0.87399
[160]	validation_0-auc:0.90252	validation_1-auc:0.87447
[180]	validation_0-auc:0.90291	validation_1-auc:0.87500
[196]	validation_0-auc:0.90291	validation_1-auc:0.87500


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=10, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, random_state=27, reg_alpha=0, reg_lambda=100,
              scale_pos_weight=1, seed=27, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [9]:
train_score = roc_auc_score(y_train, model_xgb_numeric.predict_proba(x_train)[:, 1])
test_score = roc_auc_score(y_test, model_xgb_numeric.predict_proba(x_test)[:, 1])
LB_score = roc_auc_score(y_lb, model_xgb_numeric.predict_proba(X_lb)[:, 1])

Hold_Outpd_2 = pd.DataFrame(data=[{'Train-score': train_score,
                                   'Test-score': test_score,
                                   'LB-score': LB_score}], index=['2-Hold-Out'])
Hold_Outpd_2

,Train-score,Test-score,LB-score
2-Hold-Out,0.902878,0.875026,0.862459


### Задание 2. 
Сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (`TransactionID`), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, shuffle=False, random_state=1)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, train_size=0.50, shuffle=False, random_state=1)

In [11]:
model_xgb_numeric = xgb.XGBClassifier(**params)
model_xgb_numeric.fit(X=x_train, y=y_train,
                      eval_set=[(x_train, y_train), (x_valid, y_valid)],
                      early_stopping_rounds=30,
                      eval_metric="auc",
                      verbose=20)

[0]	validation_0-auc:0.59781	validation_1-auc:0.60969
[20]	validation_0-auc:0.81385	validation_1-auc:0.82929
[40]	validation_0-auc:0.87183	validation_1-auc:0.85479
[60]	validation_0-auc:0.88592	validation_1-auc:0.86884
[80]	validation_0-auc:0.89107	validation_1-auc:0.87467
[100]	validation_0-auc:0.89542	validation_1-auc:0.87665
[120]	validation_0-auc:0.89897	validation_1-auc:0.87870
[140]	validation_0-auc:0.90179	validation_1-auc:0.88095
[160]	validation_0-auc:0.90252	validation_1-auc:0.88146
[180]	validation_0-auc:0.90291	validation_1-auc:0.88185
[196]	validation_0-auc:0.90291	validation_1-auc:0.88185


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=10, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=4, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, random_state=27, reg_alpha=0, reg_lambda=100,
              scale_pos_weight=1, seed=27, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [12]:
train_score = roc_auc_score(y_train, model_xgb_numeric.predict_proba(x_train)[:, 1])
test_score = roc_auc_score(y_test, model_xgb_numeric.predict_proba(x_test)[:, 1])
valid_score = roc_auc_score(y_valid, model_xgb_numeric.predict_proba(x_valid)[:, 1])
LB_score = roc_auc_score(y_lb, model_xgb_numeric.predict_proba(X_lb)[:, 1])

Hold_Outpd_3 = pd.DataFrame(data=[{'Train-score': train_score,
                                   'Valid-score': valid_score,
                                   'Test-score': test_score,
                                   'LB-score': LB_score}], index=['3-Hold-Out'])
Hold_Outpd_3

,Train-score,Valid-score,Test-score,LB-score
3-Hold-Out,0.902878,0.881866,0.869149,0.862459


### Задание 3. Построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

### Задание 4. Выполнить **Adversarial Validation**, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

### Задание 5. Сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных. Оценить качество на ЛБ, сделать выводы.

### Задание 6 (опциональное). Сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.

### Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).